# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Loading-libraries" data-toc-modified-id="Loading-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Loading libraries</a></div><div class="lev1 toc-item"><a href="#Resnet-model" data-toc-modified-id="Resnet-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Resnet model</a></div><div class="lev1 toc-item"><a href="#Training-400x300" data-toc-modified-id="Training-400x300-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Training 400x300</a></div><div class="lev1 toc-item"><a href="#Training-600x450" data-toc-modified-id="Training-600x450-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Training 600x450</a></div><div class="lev1 toc-item"><a href="#Predicting" data-toc-modified-id="Predicting-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Predicting</a></div>

# Loading libraries

In [73]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Sequential, load_model, Model
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.resnet50 import ResNet50
import keras, tensorflow as tf, numpy as np, gym, sys, copy, argparse, random
from keras_tqdm import TQDMNotebookCallback

from datetime import datetime
import os

import numpy as np
import pandas as pd
import math

# Setting this as the default tensorflow session. 
keras.backend.tensorflow_backend.set_session(sess)
gpu_ops = tf.GPUOptions(allow_growth=True)
config = tf.ConfigProto(gpu_options=gpu_ops)
pd.options.display.max_rows = 40
sess = tf.Session(config=config)

# Resnet model

In [32]:
base_model = ResNet50(include_top = False, weights = 'imagenet')

Exception in thread Thread-50:
Traceback (most recent call last):
  File "E:\anaconda\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "E:\anaconda\lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "E:\anaconda\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



In [33]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation = 'relu')(x)
x = Dense(1, activation = 'sigmoid')(x)
model_final = Model(inputs=base_model.input, outputs=x)

In [34]:
model_final.compile(loss = 'binary_crossentropy',
                   optimizer = SGD(lr = 0.0001, momentum = 0.9, decay = 1e-5),
                   metrics = ['accuracy'])

In [35]:
model_final.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        input_3[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_99 (Activation)      (None, None, None, 6 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

bn3a_branch2a (BatchNormalizati (None, None, None, 1 512         res3a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_109 (Activation)     (None, None, None, 1 0           bn3a_branch2a[0][0]              
__________________________________________________________________________________________________
res3a_branch2b (Conv2D)         (None, None, None, 1 147584      activation_109[0][0]             
__________________________________________________________________________________________________
bn3a_branch2b (BatchNormalizati (None, None, None, 1 512         res3a_branch2b[0][0]             
__________________________________________________________________________________________________
activation_110 (Activation)     (None, None, None, 1 0           bn3a_branch2b[0][0]              
__________________________________________________________________________________________________
res3a_bran

                                                                 activation_117[0][0]             
__________________________________________________________________________________________________
activation_120 (Activation)     (None, None, None, 5 0           add_39[0][0]                     
__________________________________________________________________________________________________
res4a_branch2a (Conv2D)         (None, None, None, 2 131328      activation_120[0][0]             
__________________________________________________________________________________________________
bn4a_branch2a (BatchNormalizati (None, None, None, 2 1024        res4a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_121 (Activation)     (None, None, None, 2 0           bn4a_branch2a[0][0]              
__________________________________________________________________________________________________
res4a_bran

__________________________________________________________________________________________________
res4d_branch2c (Conv2D)         (None, None, None, 1 263168      activation_131[0][0]             
__________________________________________________________________________________________________
bn4d_branch2c (BatchNormalizati (None, None, None, 1 4096        res4d_branch2c[0][0]             
__________________________________________________________________________________________________
add_43 (Add)                    (None, None, None, 1 0           bn4d_branch2c[0][0]              
                                                                 activation_129[0][0]             
__________________________________________________________________________________________________
activation_132 (Activation)     (None, None, None, 1 0           add_43[0][0]                     
__________________________________________________________________________________________________
res4e_bran

__________________________________________________________________________________________________
res5b_branch2b (Conv2D)         (None, None, None, 5 2359808     activation_142[0][0]             
__________________________________________________________________________________________________
bn5b_branch2b (BatchNormalizati (None, None, None, 5 2048        res5b_branch2b[0][0]             
__________________________________________________________________________________________________
activation_143 (Activation)     (None, None, None, 5 0           bn5b_branch2b[0][0]              
__________________________________________________________________________________________________
res5b_branch2c (Conv2D)         (None, None, None, 2 1050624     activation_143[0][0]             
__________________________________________________________________________________________________
bn5b_branch2c (BatchNormalizati (None, None, None, 2 8192        res5b_branch2c[0][0]             
__________

In [36]:
datagen = ImageDataGenerator(
    rotation_range                = 20,
    width_shift_range             = 0.2,
    height_shift_range            = 0.2,
    horizontal_flip               = True)

validgen = ImageDataGenerator()

# Training 400x300

In [41]:
# 600/450 _ 500/375 _ 400/300 _ 300/225

img_width  = 400
img_height = 300

train_data_dir      = "data/train"
validation_data_dir = "data/valid"
test_data_dir       = "data/test"

batch_size_train = 16
batch_size_val   = 32

train_gen = datagen.flow_from_directory(
        directory   = train_data_dir,
        target_size = (img_height, img_width),
        batch_size  = batch_size_train,
        class_mode  = "binary",
        shuffle     = True)

val_gen = validgen.flow_from_directory(
        directory   = validation_data_dir,
        target_size = (img_height, img_width),
        batch_size  = batch_size_val,
        class_mode  = "binary",
        shuffle     = False)


train_samples      = len(train_gen.filenames)
validation_samples = len(val_gen.filenames)

Found 1742 images belonging to 2 classes.
Found 553 images belonging to 2 classes.


In [42]:
checkpoint = ModelCheckpoint("weights-iter-7-epoch-{epoch:02d}.hdf5",
                             monitor='val_acc',
                             verbose=0,
                             save_best_only=False,
                             save_weights_only=True)

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')

model_final.fit_generator(generator        = train_gen,
                          epochs           = 40, 
                          steps_per_epoch  = math.ceil(train_samples / batch_size_train), 
                          validation_data  = val_gen, 
                          validation_steps = math.ceil(validation_samples / batch_size_val), 
                          verbose          = 2,
                          callbacks        = [early_stopping, TQDMNotebookCallback(), checkpoint])

HBox(children=(IntProgress(value=0, description='Training', max=40), HTML(value='')))

Epoch 1/40


HBox(children=(IntProgress(value=0, description='Epoch 0', max=109), HTML(value='')))

 - 77s - loss: 0.1376 - acc: 0.9495 - val_loss: 0.1266 - val_acc: 0.9458
Epoch 2/40


HBox(children=(IntProgress(value=0, description='Epoch 1', max=109), HTML(value='')))

...

 - 73s - loss: 0.0660 - acc: 0.9753 - val_loss: 0.0812 - val_acc: 0.9711
Epoch 13/40


HBox(children=(IntProgress(value=0, description='Epoch 12', max=109), HTML(value='')))

 - 73s - loss: 0.0697 - acc: 0.9742 - val_loss: 0.0847 - val_acc: 0.9711
Epoch 14/40


HBox(children=(IntProgress(value=0, description='Epoch 13', max=109), HTML(value='')))

 - 73s - loss: 0.0658 - acc: 0.9794 - val_loss: 0.0848 - val_acc: 0.9693
Epoch 00014: early stopping


In [50]:
model_final.load_weights('weights-iter-7-epoch-14.hdf5')

In [60]:
model_final.evaluate_generator(val_gen, math.ceil(validation_samples / batch_size_val))

[0.10640957932009835, 0.9602169981916817]

# Training 600x450

In [70]:
# 600/450 _ 500/375 _ 400/300 _ 300/225

img_width  = 600
img_height = 450

train_data_dir      = "data/train"
validation_data_dir = "data/valid"
test_data_dir       = "data/test"

batch_size_train = 8
batch_size_val   = 16

train_gen = datagen.flow_from_directory(
        directory   = train_data_dir,
        target_size = (img_height, img_width),
        batch_size  = batch_size_train,
        class_mode  = "binary",
        shuffle     = True)

val_gen = validgen.flow_from_directory(
        directory   = validation_data_dir,
        target_size = (img_height, img_width),
        batch_size  = batch_size_val,
        class_mode  = "binary",
        shuffle     = False)


train_samples      = len(train_gen.filenames)
validation_samples = len(val_gen.filenames)

Found 1742 images belonging to 2 classes.
Found 553 images belonging to 2 classes.


In [40]:
checkpoint = ModelCheckpoint("weights-iter-8-epoch-{epoch:02d}.hdf5",
                             monitor='val_acc',
                             verbose=0,
                             save_best_only=False,
                             save_weights_only=True)

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')

model_final.fit_generator(generator        = train_gen,
                          epochs           = 40, 
                          steps_per_epoch  = math.ceil(train_samples / batch_size_train), 
                          validation_data  = val_gen, 
                          validation_steps = math.ceil(validation_samples / batch_size_val), 
                          verbose          = 2,
                          callbacks        = [early_stopping, TQDMNotebookCallback(), checkpoint])

HBox(children=(IntProgress(value=0, description='Training', max=40), HTML(value='')))

Epoch 1/40


HBox(children=(IntProgress(value=0, description='Epoch 0', max=218), HTML(value='')))

 - 135s - loss: 0.4363 - acc: 0.8031 - val_loss: 0.2731 - val_acc: 0.9005
Epoch 2/40


HBox(children=(IntProgress(value=0, description='Epoch 1', max=218), HTML(value='')))

...

 - 122s - loss: 0.0636 - acc: 0.9788 - val_loss: 0.0404 - val_acc: 0.9873
Epoch 22/40


HBox(children=(IntProgress(value=0, description='Epoch 21', max=218), HTML(value='')))

 - 123s - loss: 0.0541 - acc: 0.9805 - val_loss: 0.0435 - val_acc: 0.9855
Epoch 23/40


HBox(children=(IntProgress(value=0, description='Epoch 22', max=218), HTML(value='')))

 - 123s - loss: 0.0573 - acc: 0.9811 - val_loss: 0.0418 - val_acc: 0.9855
Epoch 00023: early stopping


In [74]:
model_final.load_weights('weights-iter-8-epoch-23.hdf5')

In [75]:
model_final.evaluate_generator(val_gen, math.ceil(validation_samples / batch_size_val))

[0.04176572169884489, 0.9855334538878843]

# Predicting

In [76]:
img_height

450

In [81]:
batch_size_test = 16

test_gen = validgen.flow_from_directory(
        directory   = test_data_dir,
        target_size = (img_height, img_width),
        batch_size  = batch_size_test,
        class_mode  = "binary",
        shuffle     = False)

test_samples       = len(test_gen.filenames)
test_samples

Found 0 images belonging to 0 classes.


0

In [82]:
preds = model_final.predict_generator(test_gen, math.ceil(test_samples / batch_size_test))
preds

[]

In [83]:
preds_filenames = test_gen.filenames
preds_filenames = [int(x.replace("unknown/", "").replace(".jpg", "")) for x in preds_filenames]
df_result = pd.DataFrame({'name': preds_filenames, 'invasive': preds[:,0]})
df_result = df_result.sort_values("name")
df_result.index = df_result["name"]
df_result = df_result.drop(["name"], axis=1)
df_result.to_csv("submission_05.csv", encoding="utf8", index=True)
from IPython.display import FileLink
FileLink('submission_05.csv')

TypeError: list indices must be integers or slices, not tuple

In [84]:
# Got 0.99016 on LB